In [3]:
import numpy as np
import random
import scipy.optimize as opt
import scipy.interpolate as intpl
import matplotlib.pyplot as plt
import pdb
import sys
from numba import jit
sys.path.insert(0, "C:\\Users\\utabo\\Documents\\GitHub\\BootCamp2019\\ProblemSets\\ECON\\Week 5")
import parameters as p
import compileparam as compp

In [4]:
#Calculate transitions:
ee=0.95
eu=1-ee
uss=0.05
uu=(uss-(1-uss)*(eu))/uss
ue=1-uu
print(ue)
p.pi=np.array([[ee,eu],[ue,uu]])
p.pi=np.array([[0.95,0.05],[0.95,0.05]])
compp.pi=np.array([[ee,eu],[ue,uu]])
compp.pi=np.array([[0.95,0.05],[0.95,0.05]])

0.9500000000000008


In [5]:
def markovsim():
    eps=np.empty([p.numsim])
    eps[0]=0
    for x in range(p.numsim-1):
        ep=np.int_(eps[x])
        eps[x+1]= np.random.uniform(0,1) <= p.pi[ep,1]
    return eps

In [6]:
np.random.seed(386758)   
epsseries=markovsim()
print(np.mean(epsseries))
epsseries=np.int_(epsseries)

0.0515


In [7]:

#====================================================#
#Problem set - Tony Smith
#====================================================#

#Get interest rate
def get_r(kbar,u,p):
    r=p.alpha*kbar**(p.alpha-1)*(1-u)**(1-p.alpha)
    return r

#get wage rate
def get_k(kbar,u,p):
    w=(1-p.alpha)*kbar**p.alpha*(1-u)**(-p.alpha)
    return w

#euler equation:
def utility(x):
    if x>0:
        utils=np.log(x)
    else:
        utils=-(10**8)
    return utils


def negV(kprime,Vcont,eps,p,tt,r,w,kk) :
    V_t_interp0 = intpl.interp1d(p.kgrid,Vcont[:,0], kind='cubic', fill_value='extrapolate')
    V_t_interp1 = intpl.interp1d(p.kgrid,Vcont[:,1], kind='cubic', fill_value='extrapolate')
    Vprime=np.empty([2])
    #Vprime[0,0]=np.interp(p.kgrid,Vcont[:,0], kprime) #good
    if tt>0:
        #pdb.set_trace()
        Vprime[0]=V_t_interp0(kprime) #good
        Vprime[1]=V_t_interp1(kprime) #bad 
        #Vprime[0]=intpl.interp1d( kprime,p.kgrid,Vcont[:,0], kind='cubic',fill_value='extrapolate') #good
        #Vprime[1]=intpl.interp1d(kprime,p.kgrid,Vcont[:,1], kind='cubic',fill_value='extrapolate')#bad 
    else:
        Vprime=np.array([0][0])
    c=(1+r-p.delta)*kk+w*eps-kprime
    EV=sum(Vprime*p.pi[eps,:]) #expected value given transition probs
#     print(Vprime.shape)
#     print(p.pi.shape)
#     print(EV.shape)
    
    value=-1*(utility(c)+p.beta*EV) #negative value of choosing kprime
    return value

#Problem that the consumer solves given kbar:
@jit
def vfi(kbar,p,epsseries):
    Vcont=np.zeros([p.numptsk,2])
    pf=np.empty([p.numptsk, 2])
    vopt=np.empty([p.numptsk, 2])
    r=get_r(kbar,u,p)
    w=get_k(kbar,u,p)
    for tt in range(0,p.maxiter):
        for ixe, eps in enumerate(p.epsgrid):
            for ixk, kk in enumerate(p.kgrid):
                #res= opt.minimize_scalar(negV,bracket=(0.00001,1),args=(Vcont,eps,p,tt,r,w,kk), method='Golden' )          
                res= opt.minimize_scalar(negV,bounds=(kk-0.0001, kk),args=(Vcont,eps,p,tt,r,w,kk), method='bounded' )
                vopt[ixk,ixe] =-res.fun
                pf[ixk,ixe] =res.x
        diff=((vopt-Vcont) ** 2).sum()        
        if diff>p.tol:
            Vcont=vopt
        else:
            print("convergence 1 achieved")
            indk=np.empty(p.numsim)
            #once we found a solution we want to simulate:
            #np.random.seed(56758)   
            #print(epsseries)
            #epsseries=markovsim()
            #epsseries=np.int_(epsseries)
            indk[0]=1
            for ixN, epsilon in enumerate(epsseries):
                indk[ixN]=np.interp(indk[ixN-1],p.kgrid, pf[:,epsilon])
                #indk[ixN]=intpl.interp1d(indk[ixN-1],p.kgrid, pf[:,epsilon], kind='cubic',fill_value='extrapolate')              
                #indk[ixN]=np.interp(p.kgrid, pf[:,epsilon],kss)
            kss=np.mean(indk)
            break
    return vopt, pf,kss,indk,epsseries


def aiyagari(kbar,p,epsseries):
    for j in range(0,p.maxiter):
        print("outer loop:")
        print(j)
        vopt, pf,kss,indk,epsseries=vfi(kbar,p,epsseries)
        print(abs(kss-kbar))
        if abs(kss-kbar)>p.tol_outer:
            
            kbar=kss
            
        else:
            print("convergence 2 achieved")
            break
    return vopt, pf,kss,indk,epsseries


In [8]:
u=0.1
#initialize Vcont
vopt_out, pf_out,kss_out,indk,epsseries=aiyagari(8,compp,epsseries)


outer loop:
0
convergence 1 achieved


IndexError: index 10 is out of bounds for axis 0 with size 10

In [ ]:
vopt_out, pf_out,kss_out,indk,epsseries=aiyagari(30,p,epsseries)
print(kss_out)
print(np.std(indk)/np.mean(indk))
print(epsseries)

In [ ]:
plt.plot(p.kgrid,pf_out[:,0])
plt.plot(p.kgrid,pf_out[:,1])
plt.show()

In [ ]:
plt.plot(p.kgrid,vopt_out[:,0])
plt.plot(p.kgrid,vopt_out[:,1])
plt.show()